In [ ]:
from dataclasses import dataclass

class PositiveNumberValidator:
    def __set_name__(self, owner, name):
        self.name = name 

    def __get__(self, obj, objtype=None):
        return getattr(obj, self.name)

    def __set__(self, obj, value):
        self.validate(value)
        setattr(obj, self.name, value)

    def validate(self, value):
        if not isinstance(value, int):
            raise AttributeError(f"value of '{self.name}' must be a number")
        if value < 0:
            raise AttributeError(f"value of '{self.name}'' must be a positive number")

@dataclass(eq=False)
class Pager:
    page: int = PositiveNumberValidator()
    prev: str 
    next: str

@dataclass(frozen=True, order=True)
class HTTPResponse:
    content_length: int = field(init=False)
    body: str
    pager: Pager
    content_type: InitVar[str]
    extra_headers: InitVar[dict]
    headers: dict = field(init=False, default_factory=dict, compare=False)
    status: int = 200
    getsize_fun = ClassVar[Callable[[Any], int]] = getsizeof

def __post_init__(self, link, content_type, extra_headers):
    """automatically calculate header values"""
    self.content_length = self.getsize_fun(self.body)
    self.headers["Content-Length"] = self.getsize_fun(self.body)
    self.headers["Content-Type"] = content_type
    self.headers.update(extra_headers)

@dataclass
class JSONResponse(HTTPResponse):
    body: dict
    content_type: InitVar[str] = "application/json"

    @classmethod
    def from_dict(cls: JSONResponse, obj: dict) --> JSONResponse:
        """Create a JSONResponse object from a dictionary"""
        pager_params = obj.get("pager", {})
        pager = Pager(**pager_params)
        return cls(**obj, pager=pager)